### Experimental Comparison Notebook

In [3]:
# Imports
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from tetris_main import *
from utils import *
from library.genetic_algorithm import gao
import pickle

##### Parameter Setting

In [47]:
# Set the parameters here for the configurations to compare, then run the entire notebook!
grid_shape = (6, 8)
pieces_list = "J T S O Z T O I Z J J Z".split()

# Parameters for GA. These are the parameters that must be the same for this type of comparison.
# (According to the booklet)
pop_size = 5
iterations = 2  # The number of runs | iterations.
generations = 5

# These are the parameters that correspond and differentiate configurations to test.
N_ELITES = 2

GA_config_A = {
    "name": "A",
    "n_elites": N_ELITES,
    "params": {
        "generations": generations,
        "selection_type": "tournament",
        "tournament_size": 2,
        "crossover_type": "pmx",
        "p_crossover": 0.1,
        "mutation_type": "swap",
        "p_mutation": 0.05,
        "n_mutations": 1,
        "p_rot_mut": 0.1,
        "p_adoption": 0.01,
        "hc_hardstop": 0,
    },
}

GA_config_B = {
    "name": "B",
    "n_elites": N_ELITES,
    "params": {
        "generations": generations,
        "selection_type": "fps",
        "tournament_size": 2,
        "crossover_type": "pmx",
        "p_crossover": 0.1,
        "mutation_type": "inverted",
        "p_mutation": 0.05,
        "n_mutations": 1,
        "p_rot_mut": 0.1,
        "p_adoption": 0.01,
        "hc_hardstop": 0,
    },
}

# What GA configs to test.
GA_configs_list = [GA_config_A, GA_config_B]

# This is optional setting, since the default will be zero for all.
rotations = ["0" for i in range(len(pieces_list))]

master_dict = {
    "general": {
        "pop_size": pop_size,
        "generations": generations,
        "grid_shape": grid_shape,
        "iterations": iterations,
        "pieces_list": pieces_list,
    },
    **{config["name"]: config for config in GA_configs_list},
}


---
##### Initial Visualization

In [48]:
# This cell just makes an initial visualization of the grid and pieces.

# This will generate the individual automatically for the just the initial visualization.
vis_individual = [x+y for x,y in list(zip(pieces_list,rotations))]

grid, pieces_coordinates = tetrimino_fitter(vis_individual, grid_shape)
plot_grid(
    ind_fitness = 0,
    grid=grid,
    pieces_coordinates=pieces_coordinates,
    save_html_name= None,
    save_png_name= None,
    marker_size= 50,
    width= grid_shape[0]*70,
    height= grid_shape[1]*50
)

---
#### GA algorithm runs

In [14]:
# Initialize individuals and the population.
individuals = [generate_individual(pieces_list, grid_shape) for i in range(pop_size)]
pop = Population(
    individuals, "max", n_elites=N_ELITES, valid_set=pieces_list, grid_shape=grid_shape
)

# Initialize a dict where we store fitness history for each run.
fitness_dict = {}
# Initialize a dict for ABF for configs tested.
ABF_dict = {}

# Initialize a list to hold the collection of the best individuals.
best_individuals = {config["name"]: [] for config in GA_configs_list}



In [15]:
for GA_config in GA_configs_list:
    for iteration in range(iterations):
        pop = Population(individuals, "max", GA_config["n_elites"], pieces_list, grid_shape)
        gao.GAO(
            pop,
            **GA_config["params"],
        )
        # save fitness history each iteration.
        fitness_dict[iteration] = pop.fitness_history

        # Save the best individual of the iteration. With this construction, they will be easy to find later.
        best_individuals[GA_config["name"]].append(pop.elites[0])

    # Build a dataframe / table with fitness per generation.
    df = pd.DataFrame(fitness_dict)

    # Record the Average Best Fitness in the respective dict.
    ABF_dict[GA_config["name"]]=df.mean(axis=1)

---
#### Average Best Fitness Comparison

In [19]:
# Output a DF with the ABF comparison. 
ABF_df = np.round(pd.DataFrame(ABF_dict)).astype(int)

In [25]:
# Save 
master_dict["general"]["ABF_df"] = ABF_df

for k, v in best_individuals.items():
    master_dict[k]["best_individuals"] = v

with open("configs/fsx_master_dict.pkl", "wb") as f:
    pickle.dump(master_dict, f)

In [44]:
# Visualize comparison of ABF for different configurations.
px.line(ABF_df,labels={"index": "Generation", "value": "Average Best Fitness","variable":"Configuration"}, template="plotly")

---
#### Successful Runs / Performed Runs

In [35]:
# Output a DF with the best individuals per run and configuration.
best_ind_df = pd.DataFrame(best_individuals)

# The same DF with the fitness values only.
best_ind_df_fit = best_ind_df.applymap(lambda x:x.fitness).astype(int)

# Register max possible fitness, as well as best fitness achieved.
max_fitness = int(np.sum(grid.size*100))
best_fitness = max(best_ind_df_fit.max())
print(f"The max possible fitness was {max_fitness}.\nThe best fitness achieved was {best_fitness}.")

# Print the SR / PR ratio to 2 decimals.
print(f"Total runs: {iterations}\nRatio of Successful Runs / Performed Runs:")
# Pandas count function counts for each column (skipping NAs) and outputs a series, which is useful for this purpose.
print(np.round(best_ind_df[best_ind_df_fit==max_fitness].count()/iterations,2))
if max_fitness != best_fitness:
    print("Max fitness not achieved. Using the same ratio for best fitness.")
    print(np.round(best_ind_df[best_ind_df_fit==best_fitness].count()/iterations,2))

The max possible fitness was 4800.
The best fitness achieved was 4380.
Total runs: 30
Ratio of Successful Runs / Performed Runs:
A    0.0
B    0.0
dtype: float64
Max fitness not achieved. Using the same ratio for best fitness.
A    0.00
B    0.07
dtype: float64


In [36]:
# This is a pandas way of getting the best indidividuals for visualization.
best_inds = best_ind_df[best_ind_df_fit==best_fitness]

# Transform it into lists, skipping Nones.
best_inds = best_inds.apply(lambda x:x.dropna().to_list())

In [37]:
best_inds

A                                            []
B    [Individual <4380.0>, Individual <4380.0>]
dtype: object

---
#### Visualize an example of the best individual.

In [45]:
# The first element will be as good as any other element.
try:
    best_ind = best_inds.iloc[0][0]
except:
    best_ind = best_inds.iloc[1][0]
    
grid, pieces_coordinates = tetrimino_fitter(best_ind.representation, grid_shape)

plot_grid(
    ind_fitness=best_ind.fitness,
    grid=grid,
    pieces_coordinates=pieces_coordinates,
    save_html_name= None,
    save_png_name= None,
    marker_size= 50,
    width= grid_shape[0]*70,
    height= grid_shape[1]*50
)

In [46]:
best_ind.representation

['S3', 'Z2', 'J3', 'O3', 'T3', 'O3', 'I0', 'Z1', 'J1', 'T0', 'J0', 'Z0']